# Финальное предсказание

Были получены модели, которые максимизируют качество решения той или иной постановки задачи (классификация, регрессия).

In [1]:
import numpy as np
import pandas as pd

In [2]:
def clear_text_data(frame):
    """Производит очистку текста от лишних сиволов (всего кроме букв и цифр)"""
    new_frame = frame.copy()
    for col in ['answer1', 'answer2', 'answer3']:
        new_frame[col] = new_frame[col].str.lower().str.replace('[^\w\s]', '', regex=True)
    return new_frame

In [3]:
def lemmatize_text(text):
    """Производит лемматизацию токенов в тексте"""
    words = text.split()  # Разбиваем текст на слова
    lemmas = [morph.parse(word)[0].normal_form for word in words]  # Получаем нормальную форму слова
    return ' '.join(lemmas)  # Объединяем леммы в строку

In [4]:
def get_best_threshold(model, X_train, y_train):
    """Поиск лучшего порога отсечения, который максимизирует метрику f1_macro"""
    y_probs = model.predict_proba(X_train)[:, 1]

    thresholds = np.arange(0.0, 1.0, 0.01)
    f1_scores = []
    
    for threshold in thresholds:
        y_pred = (y_probs >= threshold).astype(int)
        f1 = f1_score(y_train, y_pred, average='macro')
        f1_scores.append(f1)
    
    optimal_threshold = thresholds[np.argmax(f1_scores)]
    optimal_f1 = max(f1_scores)
    return optimal_threshold

In [5]:
df = pd.read_csv('data.csv')

In [6]:
df = df.fillna('')

In [7]:
for i in range(3):
    df[f'answer{i+1}_len'] = df[f'answer{i+1}'].map(len)

In [8]:
df.head(3)

,id,answer1,score1,answer2,score2,answer3,score3,result,answer1_len,answer2_len,answer3_len
0,train_0,для анализа массивов данных необходимых в работе,2.0,для анализа массивов данных необходимых в работе,2.0,"стараюсь всегда брать задачи, выполнение котор...",2.0,6.0,48,48,548
1,train_1,Буду использовать полученные знания в работе д...,2.0,Автоматизирую процесс сбора данных и дальнейше...,2.0,Задача по анализу кода и содержанию пакетов - ...,1.5,5.5,139,113,320
2,train_2,хочу стать топовым программистом во всём мире ...,1.5,изучаю программирование,1.5,-,0.0,3.0,63,23,1


In [9]:
new_df = pd.read_csv('new_data.csv')  # новыне данные

In [10]:
new_df.head(3)

,id,answer1,score1,answer2,score2,answer3,score3,result
0,val_0,Что бы эффективно выполнять задачи по поддержк...,NaN,Писать скрипты для обслуживания парка серверов,NaN,Помог заказчику найти сервера с аномальной акт...,NaN,NaN
1,val_1,"Необходимо получить данные знания, чтобы измен...",NaN,"Надеюсь, пригодятся в работе.",NaN,Весь год делала вручную расчетные документы в ...,NaN,NaN
2,val_2,Ученье свет! Хочу повысить уровень своих компе...,NaN,В первую очередь я собираюсь использовать полу...,NaN,Была ситуация на позапрошлые новогодние праздн...,NaN,NaN


In [11]:
for i in range(3):
    new_df[f'answer{i+1}_len'] = new_df[f'answer{i+1}'].map(len)

In [12]:
new_df.head(3)

,id,answer1,score1,answer2,score2,answer3,score3,result,answer1_len,answer2_len,answer3_len
0,val_0,Что бы эффективно выполнять задачи по поддержк...,NaN,Писать скрипты для обслуживания парка серверов,NaN,Помог заказчику найти сервера с аномальной акт...,NaN,NaN,79,46,84
1,val_1,"Необходимо получить данные знания, чтобы измен...",NaN,"Надеюсь, пригодятся в работе.",NaN,Весь год делала вручную расчетные документы в ...,NaN,NaN,88,29,152
2,val_2,Ученье свет! Хочу повысить уровень своих компе...,NaN,В первую очередь я собираюсь использовать полу...,NaN,Была ситуация на позапрошлые новогодние праздн...,NaN,NaN,366,519,673


Произведем очистку текста от лишних символов и лемматизацию:

In [13]:
import pymorphy3
morph = pymorphy3.MorphAnalyzer() # для лемматизации

In [14]:
df = clear_text_data(df)
df.loc[:, ['answer1', 'answer2', 'answer3']] = df[['answer1', 'answer2', 'answer3']].map(lemmatize_text)

new_df = clear_text_data(new_df)
new_df.loc[:, ['answer1', 'answer2', 'answer3']] = new_df[['answer1', 'answer2', 'answer3']].map(lemmatize_text)

In [15]:
df.head(1)

,id,answer1,score1,answer2,score2,answer3,score3,result,answer1_len,answer2_len,answer3_len
0,train_0,для анализ массив данные необходимый в работа,2.0,для анализ массив данные необходимый в работа,2.0,стараться всегда брать задача выполнение котор...,2.0,6.0,48,48,548


In [16]:
new_df.head(1)

,id,answer1,score1,answer2,score2,answer3,score3,result,answer1_len,answer2_len,answer3_len
0,val_0,что бы эффективно выполнять задача по поддержк...,NaN,писать скрипт для обслуживание парк сервер,NaN,помочь заказчик найти сервер с аномальный акти...,NaN,NaN,79,46,84


In [17]:
# данные для предсказания на неразмеченных данных
X_val_df = new_df[ ['answer1', 'answer2', 'answer3', 'answer1_len', 'answer2_len', 'answer3_len'] ]

In [18]:
X_val_df.head(3)

,answer1,answer2,answer3,answer1_len,answer2_len,answer3_len
0,что бы эффективно выполнять задача по поддержк...,писать скрипт для обслуживание парк сервер,помочь заказчик найти сервер с аномальный акти...,79,46,84
1,необходимо получить дать знание чтобы изменить...,надеяться пригодиться в работа,весь год делать вручную расчётный документ в ф...,88,29,152
2,учение свет хотеть повысить уровень свой компе...,в первый очередь я собираться использовать пол...,быть ситуация на позапрошлый новогодний праздн...,366,519,673


### Предсказание финальных классов

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split, cross_val_score

import optuna

In [20]:
nltk.download('stopwords') # загрузка стоп-слов
vectorizer = TfidfVectorizer(stop_words=stopwords.words('russian')) # векторизатор TF-IDF

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\matve\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Используем несколько моделей TF-IDF + LogReg, которые показали лучшее качество классификации:

In [21]:
from sklearn.metrics import classification_report, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [22]:
X, y = df.drop(columns=['id', 'result', 'score1', 'score2', 'score3']), df[['score1', 'score2', 'score3']]
y = (y > 1.5).astype(int) # сводим к меткам классов

In [23]:
def train_model(X_train, X_test, y_train, y_test, X_val):
    """Обучение модели для предсказания оценки для ответа на котнкретный вопрос"""
    def objective(trial): 
        model = LogisticRegression(
            C=trial.suggest_float('C', 1e-4, 1e4, log=True),
            penalty=trial.suggest_categorical('penalty', ['l1', 'l2']),
            class_weight=trial.suggest_categorical('class_weight', [None, 'balanced']),
            solver='liblinear'
        )
        
        # Обучение модели
        model.fit(X_train, y_train)
        
        # Оцениваем модель с помощью кросс-валидации
        return cross_val_score(model, X_train, y_train, cv=5, scoring='f1_macro').mean()


    sampler = optuna.samplers.RandomSampler(seed=42)
    study = optuna.create_study(sampler=sampler, direction='maximize')
    study.optimize(objective, n_trials=300)

    # Обучение модели с лучшими гиперпараметрами
    best_model = LogisticRegression(**study.best_params, solver='liblinear')
    best_model.fit(X_train, y_train)

    # Выбираем оптимальный порог отсечения
    optimal_threshold = get_best_threshold(best_model, X_train, y_train)

    # Делаем предсказание для новых данных
    y_val_pred = best_model.predict(X_val)
    
    return y_val_pred

In [24]:
predictions = pd.DataFrame() # датафрейм с предсказаниями для ответов в каждой колонке

for i in range(3):
    cols = [f'answer{i+1}', f'answer{i+1}_len']

    # Выбираем ответ на конкретный вопрос
    X_df_tmp, y_df_tmp = X[cols], y[f'score{i+1}']
    X_train, X_test, y_train, y_test = train_test_split(X_df_tmp, y_df_tmp, stratify=y_df_tmp, test_size=0.3, random_state=42)
    X_val = X_val_df[cols]

    # Производим векторизацию
    lens_train = X_train[f'answer{i+1}_len'].to_numpy().reshape(-1, 1) # сохраняем столбец с весами, чтобы добавить его к признакам
    lens_test = X_test[f'answer{i+1}_len'].to_numpy().reshape(-1, 1)
    lens_val = X_val[f'answer{i+1}_len'].to_numpy().reshape(-1, 1)
    X_train = np.hstack([vectorizer.fit_transform(X_train[f'answer{i+1}']).toarray(), lens_train]) # векторизация и склеивание со столбцом длин
    X_test = np.hstack([vectorizer.transform(X_test[f'answer{i+1}']).toarray(), lens_test])
    X_val = np.hstack([vectorizer.transform(X_val[f'answer{i+1}']).toarray(), lens_val])

    y_val_pred = train_model(X_train, X_test, y_train, y_test, X_val)
    predictions[f'pred_score{i+1}'] = y_val_pred

[I 2024-10-14 12:28:32,678] A new study created in memory with name: no-name-1698ff18-ee98-4f78-9e1d-206b57f4908e
[I 2024-10-14 12:28:32,739] Trial 0 finished with value: 0.5945097348909607 and parameters: {'C': 0.09915644566638401, 'penalty': 'l1', 'class_weight': None}. Best is trial 0 with value: 0.5945097348909607.
[I 2024-10-14 12:28:32,799] Trial 1 finished with value: 0.36159026011788076 and parameters: {'C': 0.0017699302940633358, 'penalty': 'l2', 'class_weight': 'balanced'}. Best is trial 0 with value: 0.5945097348909607.
[I 2024-10-14 12:28:32,847] Trial 2 finished with value: 0.3859604571013967 and parameters: {'C': 0.00014610865886287216, 'penalty': 'l1', 'class_weight': None}. Best is trial 0 with value: 0.5945097348909607.
[I 2024-10-14 12:28:32,900] Trial 3 finished with value: 0.6787950917379586 and parameters: {'C': 0.0029324868872723803, 'penalty': 'l2', 'class_weight': None}. Best is trial 3 with value: 0.6787950917379586.
[I 2024-10-14 12:28:32,962] Trial 4 finished

In [25]:
predictions.head(3)

,pred_score1,pred_score2,pred_score3
0,1,0,0
1,0,0,1
2,1,1,1


"Вставим" полученные предсказания в исходный датафрейм:

In [26]:
source_df = pd.read_excel('dataset.xlsx')

In [27]:
source_df.head(3)

,id,Почему именно вы должны стать студентом в Школе программирования? (Свободный ответ),баллы,Как вы будете применять полученные знания и навыки? (Свободный ответ),баллы.1,"Расскажите про ситуацию (рабочую), где вы справились/взялись за задачу сверх своих обязанностей. Почему? (Свободный ответ)",баллы,итог
0,train_0,для анализа массивов данных необходимых в работе,2.0,для анализа массивов данных необходимых в работе,2.0,"стараюсь всегда брать задачи, выполнение котор...",2.0,6.0
1,train_1,Буду использовать полученные знания в работе д...,2.0,Автоматизирую процесс сбора данных и дальнейше...,2.0,Задача по анализу кода и содержанию пакетов - ...,1.5,5.5
2,train_2,хочу стать топовым программистом во всём мире ...,1.5,изучаю программирование,1.5,-,0.0,3.0


In [28]:
cols = ['баллы', 'баллы.1', 'баллы ']

source_df.iloc[651:, [2, 4, 6]] = 0
source_df.loc[:, cols] = source_df.loc[:, cols].map(lambda x: 1 if x > 1.5 else 0) 

In [29]:
for col in cols:
    source_df[col] = source_df[col].astype(int)

In [30]:
source_df.head(3)

,id,Почему именно вы должны стать студентом в Школе программирования? (Свободный ответ),баллы,Как вы будете применять полученные знания и навыки? (Свободный ответ),баллы.1,"Расскажите про ситуацию (рабочую), где вы справились/взялись за задачу сверх своих обязанностей. Почему? (Свободный ответ)",баллы,итог
0,train_0,для анализа массивов данных необходимых в работе,1,для анализа массивов данных необходимых в работе,1,"стараюсь всегда брать задачи, выполнение котор...",1,6.0
1,train_1,Буду использовать полученные знания в работе д...,1,Автоматизирую процесс сбора данных и дальнейше...,1,Задача по анализу кода и содержанию пакетов - ...,0,5.5
2,train_2,хочу стать топовым программистом во всём мире ...,0,изучаю программирование,0,-,0,3.0


In [31]:
source_df.tail(3)

,id,Почему именно вы должны стать студентом в Школе программирования? (Свободный ответ),баллы,Как вы будете применять полученные знания и навыки? (Свободный ответ),баллы.1,"Расскажите про ситуацию (рабочую), где вы справились/взялись за задачу сверх своих обязанностей. Почему? (Свободный ответ)",баллы,итог
799,val_148,"Моему отделу однозначно будет полезно, если у ...",0,"Для решения рабочих задач, связанных со сложны...",0,Помимо стандартного расчета определенных пока...,0,NaN
800,val_149,Необходимо освоить новый язык для повышения св...,0,"Необходимо овладеть новыми знаниями, которые о...",0,Удалённая работа и работа на горячей линии тех...,0,NaN
801,val_150,Желаю вспомнить навыки программирования на pyt...,0,Знания позволят автоматизировать работу. Напри...,0,Описывал ответственным проблемы по автоматичес...,0,NaN


In [32]:
predictions.shape

(151, 3)

In [33]:
source_df.dtypes

id                                                                                                                             object
Почему именно вы должны стать студентом в Школе программирования? (Свободный ответ)                                            object
баллы                                                                                                                           int32
Как вы будете применять полученные знания и навыки? (Свободный ответ)                                                          object
баллы.1                                                                                                                         int32
Расскажите про ситуацию (рабочую), где вы справились/взялись за задачу сверх своих обязанностей. Почему? (Свободный ответ)     object
баллы                                                                                                                           int32
итог                                                          

In [34]:
source_df.iloc[651:, [2, 4, 6]] = predictions

In [35]:
source_df.tail(3)

,id,Почему именно вы должны стать студентом в Школе программирования? (Свободный ответ),баллы,Как вы будете применять полученные знания и навыки? (Свободный ответ),баллы.1,"Расскажите про ситуацию (рабочую), где вы справились/взялись за задачу сверх своих обязанностей. Почему? (Свободный ответ)",баллы,итог
799,val_148,"Моему отделу однозначно будет полезно, если у ...",1,"Для решения рабочих задач, связанных со сложны...",1,Помимо стандартного расчета определенных пока...,1,NaN
800,val_149,Необходимо освоить новый язык для повышения св...,0,"Необходимо овладеть новыми знаниями, которые о...",1,Удалённая работа и работа на горячей линии тех...,1,NaN
801,val_150,Желаю вспомнить навыки программирования на pyt...,1,Знания позволят автоматизировать работу. Напри...,1,Описывал ответственным проблемы по автоматичес...,0,NaN


In [36]:
source_df.isna().sum()

id                                                                                                                              0
Почему именно вы должны стать студентом в Школе программирования? (Свободный ответ)                                             0
баллы                                                                                                                           0
Как вы будете применять полученные знания и навыки? (Свободный ответ)                                                           1
баллы.1                                                                                                                         0
Расскажите про ситуацию (рабочую), где вы справились/взялись за задачу сверх своих обязанностей. Почему? (Свободный ответ)      0
баллы                                                                                                                           0
итог                                                                                      

In [37]:
source_df['итог '] = source_df['баллы'] + source_df['баллы.1'] + source_df['баллы ']

In [38]:
source_df.tail(3)

,id,Почему именно вы должны стать студентом в Школе программирования? (Свободный ответ),баллы,Как вы будете применять полученные знания и навыки? (Свободный ответ),баллы.1,"Расскажите про ситуацию (рабочую), где вы справились/взялись за задачу сверх своих обязанностей. Почему? (Свободный ответ)",баллы,итог
799,val_148,"Моему отделу однозначно будет полезно, если у ...",1,"Для решения рабочих задач, связанных со сложны...",1,Помимо стандартного расчета определенных пока...,1,3
800,val_149,Необходимо освоить новый язык для повышения св...,0,"Необходимо овладеть новыми знаниями, которые о...",1,Удалённая работа и работа на горячей линии тех...,1,2
801,val_150,Желаю вспомнить навыки программирования на pyt...,1,Знания позволят автоматизировать работу. Напри...,1,Описывал ответственным проблемы по автоматичес...,0,2


При такой классификации можно считать, что успешно с анкетированием справились те, кто набрал итоговый балл не меньше 2.

In [39]:
source_df.to_excel('class_result_dataset.xlsx')

### Предсказание финальных оценок

С задачей регресии оценок лучше всего справилась большая модель на CatBoost (большая, то есть оперирует общим ответом пользователя, состоящим из ответов на несколько разных вопросов).

In [40]:
from catboost import CatBoostRegressor, Pool

In [41]:
source_df = pd.read_excel('dataset.xlsx')

In [42]:
source_df.head(3)

,id,Почему именно вы должны стать студентом в Школе программирования? (Свободный ответ),баллы,Как вы будете применять полученные знания и навыки? (Свободный ответ),баллы.1,"Расскажите про ситуацию (рабочую), где вы справились/взялись за задачу сверх своих обязанностей. Почему? (Свободный ответ)",баллы,итог
0,train_0,для анализа массивов данных необходимых в работе,2.0,для анализа массивов данных необходимых в работе,2.0,"стараюсь всегда брать задачи, выполнение котор...",2.0,6.0
1,train_1,Буду использовать полученные знания в работе д...,2.0,Автоматизирую процесс сбора данных и дальнейше...,2.0,Задача по анализу кода и содержанию пакетов - ...,1.5,5.5
2,train_2,хочу стать топовым программистом во всём мире ...,1.5,изучаю программирование,1.5,-,0.0,3.0


In [43]:
source_df[cols] = np.nan   # предсказываем только итоговую оценку

In [44]:
source_df.head(3)

,id,Почему именно вы должны стать студентом в Школе программирования? (Свободный ответ),баллы,Как вы будете применять полученные знания и навыки? (Свободный ответ),баллы.1,"Расскажите про ситуацию (рабочую), где вы справились/взялись за задачу сверх своих обязанностей. Почему? (Свободный ответ)",баллы,итог
0,train_0,для анализа массивов данных необходимых в работе,NaN,для анализа массивов данных необходимых в работе,NaN,"стараюсь всегда брать задачи, выполнение котор...",NaN,6.0
1,train_1,Буду использовать полученные знания в работе д...,NaN,Автоматизирую процесс сбора данных и дальнейше...,NaN,Задача по анализу кода и содержанию пакетов - ...,NaN,5.5
2,train_2,хочу стать топовым программистом во всём мире ...,NaN,изучаю программирование,NaN,-,NaN,3.0


In [45]:
df_new = pd.read_csv('new_data.csv')
df_new.head(3)

,id,answer1,score1,answer2,score2,answer3,score3,result
0,val_0,Что бы эффективно выполнять задачи по поддержк...,NaN,Писать скрипты для обслуживания парка серверов,NaN,Помог заказчику найти сервера с аномальной акт...,NaN,NaN
1,val_1,"Необходимо получить данные знания, чтобы измен...",NaN,"Надеюсь, пригодятся в работе.",NaN,Весь год делала вручную расчетные документы в ...,NaN,NaN
2,val_2,Ученье свет! Хочу повысить уровень своих компе...,NaN,В первую очередь я собираюсь использовать полу...,NaN,Была ситуация на позапрошлые новогодние праздн...,NaN,NaN


In [46]:
df_new['general_answer'] = df_new['answer1'] + ' ' + df_new['answer2'] + ' ' + df_new['answer3']

for i in range(3):
    df_new[f'answer{i+1}_len'] = df_new[f'answer{i+1}'].map(len)

In [47]:
df_new.head(3)

,id,answer1,score1,answer2,score2,answer3,score3,result,general_answer,answer1_len,answer2_len,answer3_len
0,val_0,Что бы эффективно выполнять задачи по поддержк...,NaN,Писать скрипты для обслуживания парка серверов,NaN,Помог заказчику найти сервера с аномальной акт...,NaN,NaN,Что бы эффективно выполнять задачи по поддержк...,79,46,84
1,val_1,"Необходимо получить данные знания, чтобы измен...",NaN,"Надеюсь, пригодятся в работе.",NaN,Весь год делала вручную расчетные документы в ...,NaN,NaN,"Необходимо получить данные знания, чтобы измен...",88,29,152
2,val_2,Ученье свет! Хочу повысить уровень своих компе...,NaN,В первую очередь я собираюсь использовать полу...,NaN,Была ситуация на позапрошлые новогодние праздн...,NaN,NaN,Ученье свет! Хочу повысить уровень своих компе...,366,519,673


In [48]:
X_new = df_new[ ['general_answer', 'answer1_len', 'answer2_len', 'answer3_len'] ]
X_new.head()

,general_answer,answer1_len,answer2_len,answer3_len
0,Что бы эффективно выполнять задачи по поддержк...,79,46,84
1,"Необходимо получить данные знания, чтобы измен...",88,29,152
2,Ученье свет! Хочу повысить уровень своих компе...,366,519,673
3,Хочу научиться решать задачи с помощью програм...,126,92,500
4,"У меня есть все, чтобы стать студентом: сильно...",260,257,243


In [49]:
for i in range(3):
    df[f'answer{i+1}_len'] = df[f'answer{i+1}'].map(len)

In [50]:
df['general_answer'] = df['answer1'] + ' ' + df['answer2'] + ' ' + df['answer3']

X, y = df[['general_answer', 'answer1_len', 'answer2_len', 'answer3_len']], df['result']

# через функцию не получится из-за стратификации - поэтому придется продублировать код функции здесь
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

train_pool = Pool(data=X_train, label=y_train, text_features=['general_answer'])
val_pool = Pool(data=X_val, label=y_val, text_features=['general_answer'])
test_pool = Pool(data=X_test, label=y_test, text_features=['general_answer'])

In [51]:
best_params = {
    'iterations': 937,
    'learning_rate': 0.09507635921035062,
    'depth': 10,
    'l2_leaf_reg': 0.09846738873614563
}

# Обучение модели с лучшими гиперпараметрами
best_model = CatBoostRegressor(**best_params)
best_model.fit(train_pool, eval_set=val_pool, early_stopping_rounds=50, plot=0, verbose=1)

y_new_pred = best_model.predict(X_new)

0:	learn: 2.0031300	test: 1.9412496	best: 1.9412496 (0)	total: 1.17s	remaining: 18m 17s
1:	learn: 1.9323259	test: 1.8788992	best: 1.8788992 (1)	total: 2.09s	remaining: 16m 17s
2:	learn: 1.8692644	test: 1.8276057	best: 1.8276057 (2)	total: 3s	remaining: 15m 34s
3:	learn: 1.7981338	test: 1.7967195	best: 1.7967195 (3)	total: 3.89s	remaining: 15m 7s
4:	learn: 1.7423626	test: 1.7584531	best: 1.7584531 (4)	total: 4.78s	remaining: 14m 51s
5:	learn: 1.6895779	test: 1.7217446	best: 1.7217446 (5)	total: 5.67s	remaining: 14m 39s
6:	learn: 1.6482722	test: 1.6983384	best: 1.6983384 (6)	total: 5.86s	remaining: 12m 59s
7:	learn: 1.5980380	test: 1.6699728	best: 1.6699728 (7)	total: 6.77s	remaining: 13m 6s
8:	learn: 1.5496180	test: 1.6543156	best: 1.6543156 (8)	total: 7.69s	remaining: 13m 12s
9:	learn: 1.5103282	test: 1.6429860	best: 1.6429860 (9)	total: 8.61s	remaining: 13m 18s
10:	learn: 1.4813590	test: 1.6233999	best: 1.6233999 (10)	total: 9.54s	remaining: 13m 23s
11:	learn: 1.4485411	test: 1.611109

In [52]:
source_df = pd.read_excel('dataset.xlsx')
source_df.head()

,id,Почему именно вы должны стать студентом в Школе программирования? (Свободный ответ),баллы,Как вы будете применять полученные знания и навыки? (Свободный ответ),баллы.1,"Расскажите про ситуацию (рабочую), где вы справились/взялись за задачу сверх своих обязанностей. Почему? (Свободный ответ)",баллы,итог
0,train_0,для анализа массивов данных необходимых в работе,2.0,для анализа массивов данных необходимых в работе,2.0,"стараюсь всегда брать задачи, выполнение котор...",2.0,6.0
1,train_1,Буду использовать полученные знания в работе д...,2.0,Автоматизирую процесс сбора данных и дальнейше...,2.0,Задача по анализу кода и содержанию пакетов - ...,1.5,5.5
2,train_2,хочу стать топовым программистом во всём мире ...,1.5,изучаю программирование,1.5,-,0.0,3.0
3,train_3,"Смогу применить знания в своей професси, получ...",2.0,Буду заниматься автоматизацией и отладкой рабо...,2.5,Как и указано выше мне интересно улучшить суще...,2.5,7.0
4,train_4,Повысить уровень образования.,1.0,Использовать для автоматизации своих задач.,2.0,Не возникало рабочих сверхзадач связанных с на...,1.0,4.0


In [53]:
source_df.iloc[651:, -1] = y_new_pred
source_df[cols] = np.nan

In [54]:
source_df.head()

,id,Почему именно вы должны стать студентом в Школе программирования? (Свободный ответ),баллы,Как вы будете применять полученные знания и навыки? (Свободный ответ),баллы.1,"Расскажите про ситуацию (рабочую), где вы справились/взялись за задачу сверх своих обязанностей. Почему? (Свободный ответ)",баллы,итог
0,train_0,для анализа массивов данных необходимых в работе,NaN,для анализа массивов данных необходимых в работе,NaN,"стараюсь всегда брать задачи, выполнение котор...",NaN,6.0
1,train_1,Буду использовать полученные знания в работе д...,NaN,Автоматизирую процесс сбора данных и дальнейше...,NaN,Задача по анализу кода и содержанию пакетов - ...,NaN,5.5
2,train_2,хочу стать топовым программистом во всём мире ...,NaN,изучаю программирование,NaN,-,NaN,3.0
3,train_3,"Смогу применить знания в своей професси, получ...",NaN,Буду заниматься автоматизацией и отладкой рабо...,NaN,Как и указано выше мне интересно улучшить суще...,NaN,7.0
4,train_4,Повысить уровень образования.,NaN,Использовать для автоматизации своих задач.,NaN,Не возникало рабочих сверхзадач связанных с на...,NaN,4.0


Как я понял, при вещественных оценках используется порог 6 баллов, чтобы понять, успешно ли человек прошел анкетирование.

In [55]:
source_df.to_excel('reg_result_dataset.xlsx')